In [1]:
# libraries and settings
import os
import c3d
import glob2
import math
import numpy as np
from mayavi.mlab import *
import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.animation as animation
from matplotlib import animation, rc
from mpl_toolkits.mplot3d import Axes3D

from IPython.display import HTML, Image

%matplotlib qt

rc('animation', html='html5')

In [2]:
# try to read c3d files
folder_name = 'files_motions_589'
cd3data_path = os.path.join('data', 'motion_data', folder_name)
all_files = glob2.glob(cd3data_path+'/**/*[0-9].c3d', recursive=True)

print("Loading files....")
for files in all_files: 
    print(files)

# read c3d file
file_id = 0
reader = c3d.Reader(open(all_files[file_id], 'rb'))

Loading files....
data/motion_data/files_motions_589/pour02.c3d
data/motion_data/files_motions_589/pour04.c3d
data/motion_data/files_motions_589/pour03.c3d
data/motion_data/files_motions_589/pour01.c3d
data/motion_data/files_motions_589/pour05.c3d
data/motion_data/files_motions_589/pour06.c3d


### See information about marker points [here](https://motion-database.humanoids.kit.edu/marker_set/)

In [3]:
# function to read points from file
def read_c3d(c3d_file, objs_id_end, rh_id_start, lh_id_start):
    """
    read a c3d file and output the groups in the scene with the path of the end point
    """
    # read file
    reader = c3d.Reader(open(c3d_file, 'rb'))
    
    # reader.point_used
    point_labels = reader.point_labels
    dis = 8 # distance of the endpoint id
    
    print("\nRight arm points:")
    print(point_labels[rh_id_start:rh_id_start+10])

    print("\nLeft arm points:")
    print(point_labels[lh_id_start:lh_id_start+10])

    print("\nEnd point markers:")
    print(point_labels[rh_id_start+dis:rh_id_start+dis+2])
    print(point_labels[lh_id_start+dis:lh_id_start+dis+2])
    
    object_points_ids = list(range(objs_id_end))
    left_arm_points_ids = list(range(lh_id_start,lh_id_start+10)) #28
    right_arm_points_ids = list(range(rh_id_start,rh_id_start+10)) #38

    lh_traj_points_id = list(range(lh_id_start+dis,lh_id_start+dis+2))
    rh_traj_points_id = list(range(rh_id_start+dis,rh_id_start+dis+2))

    scene_groups_ids = [object_points_ids, left_arm_points_ids, right_arm_points_ids]
    traj_groups_ids = [lh_traj_points_id, rh_traj_points_id]
    
    # collect points of interest
    scene_groups = []
    for group in scene_groups_ids:
        group_points = []
        for point_id in group:
            point_marker = []
            for i, points, analog in reader.read_frames():
                point_marker.append(points[point_id][:3])
            group_points.append(point_marker)
        scene_groups.append(np.array(group_points))
        
    # for trajectory points that we want to trace
    traj_groups = []
    for group in traj_groups_ids:
        traj_points = []
        for point_id in group:
            point_marker = []
            for i, points, analog in reader.read_frames():
                traj_mrkrs = points[point_id][:3]
                point_marker.append(traj_mrkrs)
            traj_points.append(point_marker)
        traj_groups.append(np.array(traj_points))
    
    # we want to create path from the traj_points
    mid_points = []
    for group in traj_groups:
        point_a = group[0]
        point_b = group[1]
        mid_points.append((point_a + point_b)/2.0)

    
    
    return scene_groups, mid_points

In [6]:
# read c3d file
file_id = 0
objs_id_end = 12
lh_id_start = 21
rh_id_start = 31
# dis = 8 # distance of the endpoint id
scene_groups, mid_points = read_c3d(all_files[file_id], objs_id_end, rh_id_start, lh_id_start)

# save mid_points data
l_mid_point = mid_points[0]
r_mid_point = mid_points[1]
total_steps = len(l_mid_point)


Right arm points:
[u'subjX:RSHO                    ', u'subjX:RUPA                    ', u'subjX:RAEL                    ', u'subjX:RAOL                    ', u'subjX:RWTS                    ', u'subjX:RWPS                    ', u'subjX:RFRA                    ', u'subjX:RIFD                    ', u'subjX:RHTS                    ', u'subjX:RHPS                    ']

Left arm points:
[u'subjX:LSHO                    ', u'subjX:LUPA                    ', u'subjX:LAEL                    ', u'subjX:LAOL                    ', u'subjX:LWTS                    ', u'subjX:LWPS                    ', u'subjX:LFRA                    ', u'subjX:LIFD                    ', u'subjX:LHPS                    ', u'subjX:LHTS                    ']

End point markers:
[u'subjX:RHTS                    ', u'subjX:RHPS                    ']
[u'subjX:LHPS                    ', u'subjX:LHTS                    ']


### Visualize arm motion and path of the end point

In [7]:
grp_nos = 3
def update_lines(num, dataLines, lines):
    for i,groups in enumerate(scene_groups[:grp_nos]):
        frame = groups[:,num,:]

        x_data = frame[:,0]
        y_data = frame[:,1]
        z_data = frame[:,2]
        graph_list[i]._offsets3d = (x_data, y_data, z_data)
        
    title.set_text('Wrist Path Trace, time step={}'.format(num))
        
    for line, data in zip(lines, dataLines):
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(data[0:2, :num])
        line.set_3d_properties(data[2, :num])
    return lines

# Attaching 3D axis to the figure
fig = plt.figure()
# ax = p3.Axes3D(fig)
ax = fig.add_subplot(111, projection='3d')


# the data
data = [np.transpose(l_mid_point)]

# NOTE: Can't pass empty arrays into 3d version of plot()
lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1], linewidth=2)[0] for dat in data]

# plot initial data for scatter points
colours = ['r', 'b', 'g', 'k']
graph_list = []
for i, groups in enumerate(scene_groups[:grp_nos]):
    x_data = groups[:,0,:][:,0]
    y_data = groups[:,0,:][:,1]
    z_data = groups[:,0,:][:,2]
    graph = ax.scatter(x_data, y_data, z_data, c=colours[i])
    graph_list.append(graph)

title = ax.set_title('Wrist Path Trace')
# Setting the axes properties
ax.set_xlim3d([-1700.0, 0.0])
ax.set_xlabel('X')

ax.set_ylim3d([-600.0, 800.0])
ax.set_ylabel('Y')

ax.set_zlim3d([700.0, 1600.0])
ax.set_zlabel('Z')
ax.azim = 200
ax.elev = 30

# Creating the Animation object
ani = animation.FuncAnimation(fig, update_lines, total_steps, fargs=(data, lines),
                                   interval=0, blit=False)

plt.show()



In [ ]:
# save animation:
ani.save('data/media_outputs/'+folder_name+'_trace_'+str(file_id)+'.mp4', writer=animation.FFMpegWriter(fps=30, bitrate=1000), fps=30)
# ani.save('data/media_outputs/'+folder_name+'_trace.gif', writer='imagemagick', fps=30)

In [ ]:
# display image or animation through here:
# Image(url='data/media_outputs/basic_animation.gif')